In [178]:
import pandas as pd
import numpy as np
import catboost as cb
import os
from pathlib import Path

In [179]:
script_dir = Path.cwd()

script_dir

PosixPath('/home/nickolay/workspace/python/cs_space_ml_contest/task_g')

In [180]:
__get_next_month = {}

def get_next_month(date: str) -> str:
    if date in __get_next_month:
        return __get_next_month[date]
    else:
        next = __get_next_month[date] = str(pd.Period(date, freq="M") + 1)

        return next

## Cleanup the data

In [181]:
df_raw = pd.read_csv('history.csv')

df = df_raw.drop(columns=['ads_shown_current_day', 'ads_revenue_current_day'])

df = df.sort_values('id', ascending=True)

df = df.drop_duplicates(subset=['id', 'current_month'], keep='first')

df.reset_index(drop=True, inplace=True)

# df = df.set_index(['id', 'current_month'], drop=False)

df.sort_index()

,id,logins_current_month,country,traffic_type,platform,registration_date,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,...,offers,currency_1,currency_2,currency_3,currency_4,currency_5,currency_6,currency_7,current_avg_ping,current_month
0,1,13,81,paid,iOS,2024-06-13,100,21.373946,23.33529,265,...,0,37845,730,30,26,0,0,0,0.275935,2024-07
1,1,1,81,paid,iOS,2024-06-13,1,0.867439,8.87251,18,...,0,0,0,0,0,0,0,0,0.263144,2024-09
2,2,1,193,paid,Android,2024-06-01,0,0.000000,0.00000,0,...,0,0,0,0,0,0,0,0,0.000000,2024-10
3,3,7,180,paid,Android,2024-12-12,0,0.000000,0.00000,26,...,0,0,0,0,0,0,0,0,0.430013,2024-12
4,3,2,180,paid,Android,2024-12-12,0,0.000000,0.00000,1,...,0,0,0,0,0,0,0,0,0.374937,2025-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224127,133672,2,173,paid,Android,2024-12-26,0,0.000000,0.00000,9,...,0,0,0,0,0,0,0,0,0.223729,2025-01
224128,133673,14,49,paid,Android,2024-10-11,32,0.042859,0.00000,161,...,0,0,0,0,0,0,0,0,0.359385,2024-10
224129,133674,1,208,paid,Android,2024-05-08,0,0.000000,0.00000,0,...,0,0,0,0,0,0,0,0,0.000000,2025-01
224130,133674,2,208,paid,Android,2024-05-08,2,0.137498,0.00000,5,...,0,0,0,0,0,0,0,0,0.151158,2024-07


In [182]:
# df[ df['current_month'] == '2025-04']

df

,id,logins_current_month,country,traffic_type,platform,registration_date,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,...,offers,currency_1,currency_2,currency_3,currency_4,currency_5,currency_6,currency_7,current_avg_ping,current_month
0,1,13,81,paid,iOS,2024-06-13,100,21.373946,23.33529,265,...,0,37845,730,30,26,0,0,0,0.275935,2024-07
1,1,1,81,paid,iOS,2024-06-13,1,0.867439,8.87251,18,...,0,0,0,0,0,0,0,0,0.263144,2024-09
2,2,1,193,paid,Android,2024-06-01,0,0.000000,0.00000,0,...,0,0,0,0,0,0,0,0,0.000000,2024-10
3,3,7,180,paid,Android,2024-12-12,0,0.000000,0.00000,26,...,0,0,0,0,0,0,0,0,0.430013,2024-12
4,3,2,180,paid,Android,2024-12-12,0,0.000000,0.00000,1,...,0,0,0,0,0,0,0,0,0.374937,2025-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224127,133672,2,173,paid,Android,2024-12-26,0,0.000000,0.00000,9,...,0,0,0,0,0,0,0,0,0.223729,2025-01
224128,133673,14,49,paid,Android,2024-10-11,32,0.042859,0.00000,161,...,0,0,0,0,0,0,0,0,0.359385,2024-10
224129,133674,1,208,paid,Android,2024-05-08,0,0.000000,0.00000,0,...,0,0,0,0,0,0,0,0,0.000000,2025-01
224130,133674,2,208,paid,Android,2024-05-08,2,0.137498,0.00000,5,...,0,0,0,0,0,0,0,0,0.151158,2024-07


In [183]:
# # check multiplied entries (same id + current_month)
#
# grouped = df.groupby([ 'id', 'current_month' ], as_index=False).agg(ids = ('id', 'count'))
#
# grouped[ grouped[ 'ids' ] > 1 ]

In [184]:
df['next_month'] = (pd.PeriodIndex(df['current_month'], freq='M') + 1).astype(str)

# делаем self-merge по id и next_month = current_month
df_next = df[['id', 'current_month',
              'ads_revenue_current_month',
              'revenue_current_month',
              'logins_current_month']].rename(
    columns={
        'current_month': 'next_month',
        'ads_revenue_current_month': 'ads_revenue_next_month',
        'revenue_current_month': 'revenue_next_month',
        'logins_current_month': 'logins_next_month'
    }
)

df = df.merge(df_next, how='left', on=['id', 'next_month'])

df['log_ads_revenue_next_month'] = np.log1p(df['ads_revenue_next_month'].fillna(0))
df['log_revenue_next_month'] = np.log1p(df['revenue_next_month'].fillna(0))

# активность
df['is_active_next_month'] = (df['logins_next_month'] > 0).fillna(False)

df.drop(columns=['logins_next_month', 'next_month'], inplace=True)

In [185]:
# # add training data
# def ads_revenue_next_month(row):
#     key = (row['id'], get_next_month(row[ 'current_month' ]))
#     if key in df.index:
#         next_row = df.loc[ key ]
#
#         return next_row[ 'ads_revenue_current_month' ]
#     else:
#         return 0
#
# def revenue_next_month(row):
#     key = (row['id'], get_next_month(row[ 'current_month' ]))
#     if key in df.index:
#         next_row = df.loc[ key ]
#
#         return next_row[ 'revenue_current_month' ]
#     else:
#         return 0
#
# def is_active_next_month(row):
#     key = (row['id'], get_next_month(row[ 'current_month' ]))
#     if key in df.index:
#         next_row = df.loc[ key ]
#
#         return next_row[ 'logins_current_month' ] > 0
#     else :
#         return False
#
# df[ 'ads_revenue_next_month' ] = df.apply(ads_revenue_next_month, axis=1)
# df[ 'revenue_next_month' ] = df.apply(revenue_next_month, axis=1)
# df[ 'log_ads_revenue_next_month' ] = np.log1p(df[ 'ads_revenue_next_month' ])
# df[ 'log_revenue_next_month' ] = np.log1p(df[ 'revenue_next_month' ])
# df[ 'is_active_next_month' ] = df.apply(is_active_next_month, axis=1)

In [186]:
df[ 'months_after_reg' ] = df.apply(lambda row: (pd.Period(row[ 'current_month' ], freq="M") - pd.Period(row[ 'registration_date' ], freq="M")).n, axis=1)
df.drop(columns='registration_date', inplace=True)

In [187]:
df['win_rate'] = df['wins_pvp'] / (df['games_pvp'] + 1)
df['hard_quests_rate'] = df['hard_quests'] / (df['quests'] + 1)

In [188]:
df

,id,logins_current_month,country,traffic_type,platform,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,wins_pvp,...,current_avg_ping,current_month,ads_revenue_next_month,revenue_next_month,log_ads_revenue_next_month,log_revenue_next_month,is_active_next_month,months_after_reg,win_rate,hard_quests_rate
0,1,13,81,paid,iOS,100,21.373946,23.33529,265,205,...,0.275935,2024-07,NaN,NaN,0.0,0.0,False,1,0.770677,0.080214
1,1,1,81,paid,iOS,1,0.867439,8.87251,18,7,...,0.263144,2024-09,NaN,NaN,0.0,0.0,False,3,0.368421,0.030303
2,2,1,193,paid,Android,0,0.000000,0.00000,0,0,...,0.000000,2024-10,NaN,NaN,0.0,0.0,False,4,0.000000,0.000000
3,3,7,180,paid,Android,0,0.000000,0.00000,26,20,...,0.430013,2024-12,0.0,0.0,0.0,0.0,True,0,0.740741,0.052632
4,3,2,180,paid,Android,0,0.000000,0.00000,1,0,...,0.374937,2025-01,NaN,NaN,0.0,0.0,False,1,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224127,133672,2,173,paid,Android,0,0.000000,0.00000,9,2,...,0.223729,2025-01,NaN,NaN,0.0,0.0,False,1,0.200000,0.054054
224128,133673,14,49,paid,Android,32,0.042859,0.00000,161,90,...,0.359385,2024-10,NaN,NaN,0.0,0.0,False,0,0.555556,0.055814
224129,133674,1,208,paid,Android,0,0.000000,0.00000,0,0,...,0.000000,2025-01,NaN,NaN,0.0,0.0,False,8,0.000000,0.250000
224130,133674,2,208,paid,Android,2,0.137498,0.00000,5,1,...,0.151158,2024-07,NaN,NaN,0.0,0.0,False,2,0.166667,0.000000


# Чистка данных

In [189]:
def outliers_robust_z(s: pd.Series, k: float = 3.5) -> pd.Index:
    m = s.median(skipna=True)
    mad = (s - m).abs().median(skipna=True)                       # median{|x_t - m|}
    if mad == 0 or not np.isfinite(mad):
        return pd.Index([])
    sigma_hat = 1.4826 * mad
    z = (s - m) / sigma_hat
    return s.index[z.abs() > k]

In [190]:
df = df.copy()

df['log_revenue_current_month'] = np.log1p(df['revenue_current_month'])
df['log_ads_revenue_current_month'] = np.log1p(df['ads_revenue_current_month'])


In [191]:
df[ df['current_month'] == '2025-04' ].to_csv('predict.csv')

df = df[ df['current_month'] != '2025-04' ]

In [192]:
has_revenue_next_month = df[df['revenue_next_month'] > 0]

has_revenue_next_month

,id,logins_current_month,country,traffic_type,platform,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,wins_pvp,...,ads_revenue_next_month,revenue_next_month,log_ads_revenue_next_month,log_revenue_next_month,is_active_next_month,months_after_reg,win_rate,hard_quests_rate,log_revenue_current_month,log_ads_revenue_current_month
18,10,13,164,paid,iOS,247,13.075025,55.007869,314,191,...,26.512756,121.358247,3.314650,4.806953,True,0,0.606349,0.045317,4.025492,2.644402
83,59,3,50,organic,iOS,7,0.072389,0.000000,34,25,...,0.336345,0.580000,0.289938,0.457425,True,0,0.714286,0.043478,0.000000,0.069889
111,78,31,153,paid,Android,75,1.055468,0.000000,187,107,...,1.619926,1.386000,0.963146,0.869618,True,3,0.569149,0.097902,0.000000,0.720504
310,211,29,106,paid,Android,65,1.056081,1.393000,92,73,...,0.330513,0.693000,0.285565,0.526502,True,5,0.784946,0.105882,0.872548,0.720802
356,244,7,180,paid,Android,21,0.670987,0.000000,144,93,...,0.654616,0.693000,0.503569,0.526502,True,0,0.641379,0.046729,0.000000,0.513415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223841,133485,10,71,paid,Android,109,4.776956,0.000000,226,128,...,13.794182,3.479000,2.694234,1.499400,True,0,0.563877,0.037037,0.000000,1.753877
223864,133497,2,112,paid,iOS,32,1.479628,3.222924,82,53,...,15.251650,19.098080,2.788194,3.000624,True,0,0.638554,0.054945,1.440528,0.908109
223920,133538,11,112,organic,iOS,11,3.949728,6.532887,104,74,...,2.066304,107.834951,1.120473,4.689833,True,0,0.704762,0.045662,2.019278,1.599333
223921,133538,22,112,organic,iOS,2,2.066304,107.834951,206,127,...,0.000000,148.742129,0.000000,5.008915,True,1,0.613527,0.097938,4.689833,1.120473


In [193]:
outliers_rev_next_month = outliers_robust_z(has_revenue_next_month['log_revenue_next_month'], k=3)
outliers_rev_next_month

Index([  4115,   5090,  12371,  30598,  32070,  40561,  40562,  56319,  56321,
        58210,  63255,  63256,  81248,  81250,  81252,  81253,  92260,  98223,
        98224, 127930, 133752, 133759, 135499, 135501, 135502, 135504, 140284,
       142137, 142453, 149435, 149436, 160311, 169346, 170684, 183260, 184428,
       184432, 189203, 189204, 189842, 189844, 189845, 189846, 209947, 214216,
       215069, 215073],
      dtype='int64')

In [194]:
clean1 = df#.drop(outliers_rev_next_month)

clean1

,id,logins_current_month,country,traffic_type,platform,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,wins_pvp,...,ads_revenue_next_month,revenue_next_month,log_ads_revenue_next_month,log_revenue_next_month,is_active_next_month,months_after_reg,win_rate,hard_quests_rate,log_revenue_current_month,log_ads_revenue_current_month
0,1,13,81,paid,iOS,100,21.373946,23.33529,265,205,...,NaN,NaN,0.0,0.0,False,1,0.770677,0.080214,3.191928,3.107897
1,1,1,81,paid,iOS,1,0.867439,8.87251,18,7,...,NaN,NaN,0.0,0.0,False,3,0.368421,0.030303,2.289754,0.624568
2,2,1,193,paid,Android,0,0.000000,0.00000,0,0,...,NaN,NaN,0.0,0.0,False,4,0.000000,0.000000,0.000000,0.000000
3,3,7,180,paid,Android,0,0.000000,0.00000,26,20,...,0.0,0.0,0.0,0.0,True,0,0.740741,0.052632,0.000000,0.000000
4,3,2,180,paid,Android,0,0.000000,0.00000,1,0,...,NaN,NaN,0.0,0.0,False,1,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224127,133672,2,173,paid,Android,0,0.000000,0.00000,9,2,...,NaN,NaN,0.0,0.0,False,1,0.200000,0.054054,0.000000,0.000000
224128,133673,14,49,paid,Android,32,0.042859,0.00000,161,90,...,NaN,NaN,0.0,0.0,False,0,0.555556,0.055814,0.000000,0.041966
224129,133674,1,208,paid,Android,0,0.000000,0.00000,0,0,...,NaN,NaN,0.0,0.0,False,8,0.000000,0.250000,0.000000,0.000000
224130,133674,2,208,paid,Android,2,0.137498,0.00000,5,1,...,NaN,NaN,0.0,0.0,False,2,0.166667,0.000000,0.000000,0.128831


In [195]:
has_ads_revenue_next_month = clean1[clean1['ads_revenue_next_month'] > 0]
has_ads_revenue_next_month

,id,logins_current_month,country,traffic_type,platform,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,wins_pvp,...,ads_revenue_next_month,revenue_next_month,log_ads_revenue_next_month,log_revenue_next_month,is_active_next_month,months_after_reg,win_rate,hard_quests_rate,log_revenue_current_month,log_ads_revenue_current_month
5,4,22,106,paid,Android,11,0.077312,0.000,266,140,...,0.004090,0.0,0.004081,0.0,True,0,0.524345,0.057823,0.000000,0.074469
8,6,11,133,organic,Android,47,0.000000,0.000,145,71,...,0.036284,0.0,0.035641,0.0,True,0,0.486301,0.029703,0.000000,0.000000
9,6,27,133,organic,Android,146,0.036284,0.000,385,186,...,0.106053,0.0,0.100798,0.0,True,1,0.481865,0.065517,0.000000,0.035641
10,6,31,133,organic,Android,163,0.106053,0.000,246,125,...,0.040797,0.0,0.039987,0.0,True,2,0.506073,0.077253,0.000000,0.100798
15,8,6,46,paid,Android,112,0.108613,0.000,173,103,...,0.126118,0.0,0.118776,0.0,True,0,0.591954,0.041916,0.000000,0.103110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224093,133649,11,153,organic,Android,26,0.068609,0.000,62,41,...,0.002566,0.0,0.002562,0.0,True,0,0.650794,0.040816,0.000000,0.066358
224106,133658,24,86,paid,Android,1067,3.436036,1.386,728,485,...,0.315733,0.0,0.274394,0.0,True,0,0.665295,0.056980,0.869618,1.489761
224108,133659,12,153,paid,Android,28,0.015844,0.000,120,68,...,0.004718,0.0,0.004707,0.0,True,0,0.561983,0.052941,0.000000,0.015720
224112,133661,10,112,organic,iOS,52,1.721284,0.000,90,54,...,0.039648,0.0,0.038882,0.0,True,0,0.593407,0.040201,0.000000,1.001104


In [196]:
outliers_ads_rev_next_month = outliers_robust_z(has_ads_revenue_next_month['log_ads_revenue_next_month'], k=3.5)

outliers_ads_rev_next_month

Index([    18,    409,    410,    432,    450,    466,    534,    547,    555,
          620,
       ...
       223781, 223784, 223785, 223840, 223841, 223864, 223945, 223951, 224000,
       224011],
      dtype='int64', length=6596)

In [197]:
clean2 = clean1#.drop(outliers_ads_rev_next_month)

In [198]:
clean2

,id,logins_current_month,country,traffic_type,platform,ads_shown_current_month,ads_revenue_current_month,revenue_current_month,games_pvp,wins_pvp,...,ads_revenue_next_month,revenue_next_month,log_ads_revenue_next_month,log_revenue_next_month,is_active_next_month,months_after_reg,win_rate,hard_quests_rate,log_revenue_current_month,log_ads_revenue_current_month
0,1,13,81,paid,iOS,100,21.373946,23.33529,265,205,...,NaN,NaN,0.0,0.0,False,1,0.770677,0.080214,3.191928,3.107897
1,1,1,81,paid,iOS,1,0.867439,8.87251,18,7,...,NaN,NaN,0.0,0.0,False,3,0.368421,0.030303,2.289754,0.624568
2,2,1,193,paid,Android,0,0.000000,0.00000,0,0,...,NaN,NaN,0.0,0.0,False,4,0.000000,0.000000,0.000000,0.000000
3,3,7,180,paid,Android,0,0.000000,0.00000,26,20,...,0.0,0.0,0.0,0.0,True,0,0.740741,0.052632,0.000000,0.000000
4,3,2,180,paid,Android,0,0.000000,0.00000,1,0,...,NaN,NaN,0.0,0.0,False,1,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224127,133672,2,173,paid,Android,0,0.000000,0.00000,9,2,...,NaN,NaN,0.0,0.0,False,1,0.200000,0.054054,0.000000,0.000000
224128,133673,14,49,paid,Android,32,0.042859,0.00000,161,90,...,NaN,NaN,0.0,0.0,False,0,0.555556,0.055814,0.000000,0.041966
224129,133674,1,208,paid,Android,0,0.000000,0.00000,0,0,...,NaN,NaN,0.0,0.0,False,8,0.000000,0.250000,0.000000,0.000000
224130,133674,2,208,paid,Android,2,0.137498,0.00000,5,1,...,NaN,NaN,0.0,0.0,False,2,0.166667,0.000000,0.000000,0.128831


In [199]:
clean2.to_csv('training.csv', index=False)

### Агрегация по стране

In [200]:
df = clean2

agg_df = df.groupby('country', as_index=False).agg(
    log_revenue_next_month = ('log_revenue_next_month', 'sum'),
    log_revenue_next_month_count = ('log_revenue_next_month', 'count')
)

agg_df

,country,log_revenue_next_month,log_revenue_next_month_count
0,1,307.550159,1172
1,2,0.000000,9
2,3,0.000000,20
3,4,0.000000,2
4,5,482.837629,1213
...,...,...,...
201,204,21.155076,113
202,205,30.323826,56
203,206,0.000000,2
204,207,11.677975,82


### Сбалансированность классов

In [201]:
clean2.groupby('is_active_next_month', as_index=False).agg( count = ('id', 'count') )

,is_active_next_month,count
0,False,117722
1,True,79160
